In [1]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
from theano import *
from lasagne.layers import EmbeddingLayer, InputLayer, get_output
import lasagne
import lasagne.layers
import theano.tensor as T
import theano
import numpy as np

In [5]:
l_in = lasagne.layers.InputLayer((100,50))

In [6]:
l_hidden = lasagne.layers.DenseLayer(l_in, num_units=10)

/home/matthew/.virtualenvs/nlp-convnet/lib/python2.7/site-packages/lasagne/init.py:86: UserWarning: The uniform initializer no longer uses Glorot et al.'s approach to determine the bounds, but defaults to the range (-0.01, 0.01) instead. Please use the new GlorotUniform initializer to get the old behavior. GlorotUniform is now the default for all layers.
  warnings.warn("The uniform initializer no longer uses Glorot et al.'s "


In [7]:
l_out = lasagne.layers.DenseLayer(l_hidden, num_units=1)

In [8]:
out = lasagne.layers.get_output(l_out)

/home/matthew/.virtualenvs/nlp-convnet/lib/python2.7/site-packages/lasagne/layers/helper.py:69: UserWarning: get_all_layers() has been changed to return layers in topological order. The former implementation is still available as get_all_layers_old(), but will be removed before the first release of Lasagne. To ignore this warning, use `warnings.filterwarnings('ignore', '.*topo.*')`.
  warnings.warn("get_all_layers() has been changed to return layers in "


In [9]:
print out

Elemwise{mul,no_inplace}.0


In [22]:
target = T.ivector('y')
loss = lasagne.objectives.binary_crossentropy(out, target).mean()

In [17]:
all_params = lasagne.layers.get_all_params(l_out)

In [23]:
update = lasagne.updates.adagrad(loss, all_params, .01, .9)

In [48]:
train = theano.function(
    [l_in.input_var, target], loss, updates=update,
    allow_input_downcast=True,
)

In [36]:
target_v = np.random.randint(0,2,100)

In [42]:
target_t = np.random.randn(100, 50)

In [49]:
train(target_t, target_v)

ValueError: Input dimension mis-match. (input[0].shape[1] = 100, input[1].shape[1] = 1)
Apply node that caused the error: Elemwise{Composite{((i0 * log(i1)) + (i2 * log(i3)))}}[(0, 1)](InplaceDimShuffle{x,0}.0, Elemwise{mul,no_inplace}.0, Elemwise{sub,no_inplace}.0, Elemwise{sub,no_inplace}.0)
Inputs types: [TensorType(int32, row), TensorType(float64, matrix), TensorType(float64, row), TensorType(float64, matrix)]
Inputs shapes: [(1, 100), (100, 1), (1, 100), (100, 1)]
Inputs strides: [(400, 4), (8, 8), (800, 8), (8, 8)]
Inputs values: ['not shown', 'not shown', 'not shown', 'not shown']

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.